In [3]:
import sys
sys.path.append("../../../lhillber/ray_trace")
from importlib import reload
import ray_trace
reload(ray_trace)
from ray_trace import sound_speed, trace_bundle, Ray, GaussianBundle, Mirror, \
                      Detector, BiconvexLens, GradientIndexRegion, \
                      make_lens, SphericalSource, OpticalSystem
from time import time as wtime
from os import cpu_count
import itertools

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.path import Path
from matplotlib.patches import PathPatch
from matplotlib.collections import PatchCollection
from matplotlib.patches import Arc as ArcPatch
from scipy.interpolate import RegularGridInterpolator
from scipy.stats import binned_statistic as binstat
from scipy.integrate import trapz
from scipy.optimize import minimize

from joblib import Parallel, delayed
from joblib.externals.loky import set_loky_pickler
set_loky_pickler("dill")

In [4]:
def sagnac(time):
    # Sagnac parameters
    space = [-0.1, 0.6, -0.2, 0.5] # Working space
    arm1 = 0.5 # Horizontal arm length
    arm2 = 0.35 # Vertical arm length
    obs_distance = 0.08 # Distance of detector from BS dark-port
    mirror_size = 25e-3 # Mirror diameter

    # 1:1 telescope parameters
    target_focal_length = 0.15 # Single lens focal length
    approximate_thickness = 0.01 # Lens thickness
    lens_index = 1.5 # Lens refractive index (~1.5 for BK7 glass)

    # Optical beam parameters for each arm (CW and CCW), modeled as a bundle of rays
    Nrays = 100 # Number of rays per beam
    beam_waist = 3e-3/2 # Gaussian waist used to scale ray powers
    beam_power_ratio = 1 # CCW / CW beam powers (CW beam power taken as unity)

    # Acoustic source paramters
    reference_amplitude = 3e-6 # Boundary condition
    reference_distance = 10e-2 # Boundary condition
    frequency = 40e3 # Piezo acoustic frequency
    epicenter = [0.25 + 0.0, 0.33] # Spherical wave center
    grin_step_size = 1e-5 # Runge-Kutta step size for ray tracing in gradient index (GRIN) medium

    def temporal_profile(time): # Functional form of sound source's temporal profile, converted to traveling spherical wave
        import numpy as np
        return np.sin(2 * np.pi * frequency * time)

    def temporal_derivative(time): # Functional form of temporal profile derivative (optional: will use interpolation if not given)
        import numpy as np
        return 2 * np.pi * frequency * np.cos(2 * np.pi * frequency * time)

    source = SphericalSource(reference_amplitude, reference_distance, epicenter)
    index_func = source.make_index_func(time, temporal_profile)
    grad_index_func = source.make_grad_index_func(time, temporal_profile, temporal_derivative)

    grin = GradientIndexRegion(index_func, grin_step_size,
                               [[epicenter[0] - time * sound_speed, epicenter[0] + time * sound_speed, 200],
                                [epicenter[1] + times[0] / 2 * sound_speed, epicenter[1] + time * sound_speed, 200]],
                               grad_index_func = grad_index_func)

    # Set up mirrors
    BS = Mirror(mirror_size, [0, 0], np.pi / 4)
    M1 = Mirror(mirror_size, [arm1, BS.center[1]], np.pi / 4)
    M2 = Mirror(mirror_size, [arm1, arm2], -np.pi / 4)
    M3 = Mirror(mirror_size, [BS.center[0], arm2], np.pi / 4)
    detector = Detector(6 * beam_waist, [BS.center[0], -obs_distance], 0)

    # Set up telescope
    lens = make_lens(target_focal_length, approximate_thickness, lens_index)
    x = (lens.thickness / 2 + lens.focal_length - lens.principal_plane1 / 2) # distance from lens center to focus for thick lens
    L1 = BiconvexLens(lens.radius, lens.radius, lens.thickness, lens.index, [arm1 / 2 - x, arm2], 0.0)
    L2 = BiconvexLens(lens.radius, lens.radius, lens.thickness, lens.index, [arm1 / 2 + x, arm2], 0.0)

    # CW and CCW branches
    CW_sys = OpticalSystem(extent = space, objects = [BS, L1, L2, grin, M1, M2, M3, detector]) # include BS for CW
    CCW_sys = OpticalSystem(extent = space, objects = [L1, L2, grin, M1, M2, M3, detector]) # exclude BS for CCW
    CW_bundle = GaussianBundle(waist = beam_waist, center = [space[0], BS.center[1]], direction = [1, 0.], number = Nrays, total_power = 1)
    CCW_bundle = GaussianBundle(waist = beam_waist, center = [space[0], BS.center[1]], direction = [1, 0.], number = Nrays, total_power = beam_power_ratio)

    # Trace rays
    trace_bundle(CW_sys, CW_bundle, n_jobs = -2)
    trace_bundle(CCW_sys, CCW_bundle, n_jobs = -2)
    detector.set_bundles([CW_bundle, CCW_bundle])
    return CW_sys, detector